<a href="https://colab.research.google.com/github/MarkLimmage/AustralianSharePrices/blob/main/A3_Mark_Limmage_03.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install swifter
import swifter
import re
import json
import csv
from io import StringIO
from bs4 import BeautifulSoup
import requests
import pandas as pd
from tqdm.notebook import tqdm

import re
import nltk
nltk.download('stopwords')
nltk.download('punkt')
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

import dateutil.parser as dp
import time
import math
import concurrent.futures
import multiprocessing
from multiprocessing import Pool



Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 830 kB 14.8 MB/s 
     |████████████████████████████████| 281 kB 68.9 MB/s 
     |████████████████████████████████| 1.6 MB 66.5 MB/s 
  Created wheel for swifter: filename=swifter-1.3.4-py3-none-any.whl size=16322 sha256=ac6da233ad28b2a7b8e31893463b9edf81cba94d7605ec05f0b7197805b5cdac
  Stored in directory: /root/.cache/pip/wheels/29/a7/0e/3a8f17ac69d759e1e93647114bc9bdc95957e5b0cbfd405205
Successfully built swifter
  Attempting uninstall: psutil
    Found existing installation: psutil 5.4.8
    Uninstalling psutil-5.4.8:
      Successfully uninstalled psutil-5.4.8


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [ ]:
# Collect up the fragments of the df with the stories in it.

motley_5 = pd.DataFrame( columns=['company', 'article_url', 'category', 'pub_time', 'title', 'time', 'key_points', 'body'])

In [ ]:



n=18
path = '/content/drive/MyDrive/Colab Notebooks/MA5851/assessments/A2/motley_3_{}.csv'
frames = list()
for i in range(n):
  data = pd.read_csv(path.format( str(i) ))
  frames.append(data)

motley_5 = pd.concat(frames)


In [ ]:
motley_5

,Unnamed: 0,Unnamed: 0.1,company,article_url,category,pub_time,title,time,key_points,body
0,0,0,360,https://www.fool.com.au/2022/08/17/down-40-thi...,Technology Shares,NaN,"Down 40% this year, should you pounce on this ...",1.660695e+09,NaN,The Life360 Inc (ASX: 360) share price swung l...
1,1,1,360,https://www.fool.com.au/2022/08/16/here-are-th...,Share Market News,NaN,Here are the top 10 ASX 200 shares today,1.660632e+09,NaN,S&P/ASX 200 Index (ASX: XJO) shares pushed thr...
2,2,2,360,https://www.fool.com.au/2022/08/16/why-bhp-lif...,Share Gainers,NaN,"Why BHP, Life360, Tassal, and Temple & Webster...",1.660627e+09,NaN,"In afternoon trade, the S&P/ASX 200 Index (ASX..."
3,3,3,360,https://www.fool.com.au/2022/08/16/life360-sha...,Earnings Results,NaN,Life360 share price rises as revenue more than...,1.660610e+09,The Life360 share price is climbing 3% higher ...,The Life360 Inc (ASX: 360) share price is boun...
4,4,4,360,https://www.fool.com.au/2022/08/12/2-exciting-...,Growth Shares,NaN,2 exciting ASX tech shares to buy now accordin...,1.660293e+09,NaN,"If you’re searching for growth shares to buy, ..."
...,...,...,...,...,...,...,...,...,...,...
2055,37075,37075,ZEL,https://www.fool.com.au/2022/03/17/asx-200-asx...,Share Market News,NaN,ASX 200 (ASX:XJO) midday update: Block and Zip...,1.647479e+09,NaN,NaN
2056,37076,37076,ZEL,https://www.fool.com.au/2021/11/24/3-retail-as...,Broker Notes,NaN,3 retail ASX shares to buy right now: experts,1.637701e+09,NaN,NaN
2057,37077,37077,ZEL,https://www.fool.com.au/2021/11/16/this-leadin...,Blue Chip Shares,NaN,Leading fund manager says these blue-chip ASX ...,1.637032e+09,NaN,NaN
2058,37078,37078,ZEL,https://www.fool.com.au/2021/10/11/which-asx-3...,Share Market News,NaN,Which ASX 300 shares are making headlines on M...,1.633927e+09,NaN,The S&P/ASX 300 Index (ASX: XKO) is starting t...


In [ ]:
n=6
path = '/content/drive/MyDrive/Colab Notebooks/MA5851/assessments/A2/motley_4_{}.csv'
frames = list()
for i in range(n):
  
  data = pd.read_csv(path.format( str(i) ))
  frames.append(data)

motley_6 = pd.concat(frames)

In [ ]:
motley_6.drop(['Unnamed: 0.1', 'Unnamed: 0'], axis=1, inplace=True)
motley_5.drop(['Unnamed: 0.1', 'Unnamed: 0'], axis=1, inplace=True)

In [ ]:
motley_7 = pd.concat([motley_5, motley_6])


In [ ]:
motley_7.reset_index(drop=True, inplace=True)

In [ ]:
# count body nans
print('NaNs\n~~~~')
print('body:',motley_5['body'].isna().sum(), 
      ' | key points:', motley_5['key_points'].isna().sum(),
      ' | title:', motley_5['title'].isna().sum(),
      ' | category:', motley_5['category'].isna().sum(),
      ' | time:', motley_5['time'].isna().sum())

# count body nans
print('NaNs\n~~~~')
print('body:',motley_6['body'].isna().sum(), 
      ' | key points:', motley_6['key_points'].isna().sum(),
      ' | title:', motley_6['title'].isna().sum(),
      ' | category:', motley_6['category'].isna().sum(),
      ' | time:', motley_6['time'].isna().sum())

# count body nans
print('NaNs\n~~~~')
print('body:',motley_7['body'].isna().sum(), 
      ' | key points:', motley_7['key_points'].isna().sum(),
      ' | title:', motley_7['title'].isna().sum(),
      ' | category:', motley_7['category'].isna().sum(),
      ' | time:', motley_7['time'].isna().sum())

motley_7.shape

NaNs
~~~~
body: 27241  | key points: 32617  | title: 5769  | category: 4735  | time: 5770
NaNs
~~~~
body: 5271  | key points: 4904  | title: 0  | category: 0  | time: 0
NaNs
~~~~
body: 32512  | key points: 37521  | title: 5769  | category: 4735  | time: 5770


(42648, 8)

In [ ]:
# Save to disk

motley_7.to_csv('/content/drive/MyDrive/Colab Notebooks/MA5851/assessments/A2/motley_7.csv')

# Add in the price data as target values.
The gradient of the line joining the price at the timne of publishing and 1 week later will be used to measure gain / fall / stay.

This approach did not work. now trying to retreive proce data for specific periods based on story date.

In [ ]:
# Get the price df

path = '/content/drive/MyDrive/Colab Notebooks/MA5851/assessments/A2/a_prices.csv'

prices = pd.read_csv(path)

prices

,date,360,3DP,A200,A2M,A4N,AAA,AAC,ABB,ABY,...,UWL,VG1,VG8,VGAD,VUL,WAF,WGN,WMI,WSP,XRO
0,1577919600,3.19,0.058,111.599998,14.02,0.195,50.070000,1.105,NaN,NaN,...,1.635,2.24,2.320,75.500000,0.155,0.425,2.720,1.465,1.560,79.750000
1,1578006000,3.18,0.056,112.220001,14.03,0.200,50.070000,1.100,NaN,NaN,...,1.650,2.22,2.320,75.400002,0.155,0.470,2.700,1.470,1.565,79.720001
2,1578265200,3.00,0.054,112.349998,14.09,0.200,50.080002,1.090,NaN,NaN,...,1.620,2.22,2.310,75.190002,0.150,0.460,2.660,1.460,1.540,80.760002
3,1578351600,3.04,0.057,113.970001,14.17,0.190,50.090000,1.100,NaN,NaN,...,1.620,2.21,2.310,75.820000,0.160,0.440,2.500,1.470,1.560,82.040001
4,1578438000,3.15,0.055,113.650002,14.01,0.190,50.080002,1.100,NaN,NaN,...,1.585,2.20,2.290,75.160004,0.160,0.450,2.500,1.435,1.570,80.650002
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
666,1660780800,5.75,0.220,118.430000,4.93,0.470,50.119999,1.885,3.32,1.90,...,NaN,1.60,1.895,82.820000,8.030,1.315,1.125,1.715,1.100,90.650002
667,1660867200,5.63,0.215,118.480003,4.96,0.465,50.119999,1.895,3.18,1.99,...,NaN,NaN,NaN,82.970001,8.060,1.340,1.140,NaN,1.075,88.800003
668,1660888083,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,1.60,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
669,1660889461,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.705,NaN,NaN


### Convert dates
Since all the closing dates are stored as unix epoch at 00:00 GMT on the day of close, searching for start date should go back to previous date.

In [ ]:
!python -V

Python 3.7.13


In [ ]:
import datetime
from datetime import date
from datetime import datetime

def get_7_day_percent_change( d, stock):
  if(math.isnan(d)):
    return
  else:  

    date_time = datetime.fromtimestamp( d )
    date_d =   date_time.date()
    midnight = datetime.combine(date_d, datetime.min.time())
    midnight= midnight.timestamp()
    return midnight
  

motley_7['midnight'] = ''
motley_7['midnight'] = motley_7[['time','company']].swifter.apply(lambda x: get_7_day_percent_change( x['time'], x['company']), axis=1)   

Pandas Apply:   0%|          | 0/42648 [00:00<?, ?it/s]

In [ ]:
motley_7[['time']]

,time
0,1.660695e+09
1,1.660632e+09
2,1.660627e+09
3,1.660610e+09
4,1.660293e+09
...,...
42643,1.587338e+09
42644,1.585552e+09
42645,1.585540e+09
42646,1.584426e+09


In [ ]:
motley_7['midnight'].sample(20)

1322              NaN
40701    1.605658e+09
19148    1.636070e+09
14449    1.641514e+09
26195    1.615680e+09
15684    1.529626e+09
20145    1.599696e+09
29544    1.563235e+09
18136    1.599437e+09
21121    1.641514e+09
12687    1.629072e+09
21281    1.650413e+09
15712    1.518653e+09
1454              NaN
15110    1.534378e+09
11609             NaN
29294    1.629677e+09
9686              NaN
1138     1.644192e+09
19824    1.591142e+09
Name: midnight, dtype: float64

In [ ]:
def convert_close_time( d ):
  if(math.isnan(d)):
    return
  if (datetime.fromtimestamp(d).hour ==23):
    d = d+3600
  date_time = datetime.fromtimestamp( d )
  date_d =   date_time.date()
  midnight = datetime.combine(date_d, datetime.min.time()).timestamp()
  return midnight

prices['new_date'] = ''

for index, row in prices.iterrows():
  prices.loc[index, 'new_date'] = convert_close_time(row['date'])
  


In [ ]:
isin = 0
notin=0

for index, row in motley_7.iterrows():
  if(row['midnight'] in prices['new_date']):
    isin = isin + 1
  else:
    notin = notin + 1
print('is in:', isin, 'not in:', notin)

is in: 0 not in: 42648


In [ ]:
print( prices['new_date'].sample(20),motley_7['midnight'].sample(20) )

201    1602720000.0
469    1635984000.0
624    1655769600.0
399    1627516800.0
243    1607904000.0
374    1624492800.0
38     1582675200.0
294    1614556800.0
204    1603152000.0
508    1640908800.0
322    1618185600.0
588    1651190400.0
606    1653436800.0
641    1657756800.0
342    1620604800.0
32     1581984000.0
560    1647475200.0
587    1651104000.0
279    1612742400.0
71     1586908800.0
Name: new_date, dtype: object 22804    1.585267e+09
42020    1.649894e+09
4702     1.642205e+09
34344    1.601683e+09
13539    1.589242e+09
7205              NaN
5996              NaN
28825    1.652832e+09
3917              NaN
40309    1.450224e+09
2689     1.624752e+09
26775    1.496794e+09
41049    1.582157e+09
4106              NaN
26998    1.659658e+09
1811              NaN
27196    1.628467e+09
30957    1.646438e+09
3763              NaN
41332    1.646266e+09
Name: midnight, dtype: float64


In [ ]:
  date_time = datetime.fromtimestamp(  1.602547e+09 )
  date_d =   date_time.date()
  midnight = datetime.combine(date_d, datetime.min.time()).timestamp()
  print(midnight)

1602460800.0


In [ ]:
# Check how many prices there are for story days
isin = 0
notin = 0
for index, row in motley_7.sample(20).iterrows():
  if(row['midnight'] in prices['new_date']):
    isin = isin + 1
  else:
    notin = notin + 1
print('is in:', isin, 'not in:', notin)



is in: 0 not in: 20


In [ ]:
date

datetime.date

In [ ]:
import datetime

def get_7_day_percent_change( date, stock):
  if(math.isnan(date)):
    return
  #There are ~5k records with no time   
  #find the most recent price back from given date
  y1 = prices.loc[prices['date'] == date, stock]
  i=0
  while( len(y1)==0 and i < 250000 ):
    date = round((date - 100),-2)
    y1 = prices.loc[prices['date'] == date, stock]
    i = i + 1
  if(len(y1)==0):
    return
  orig = datetime.datetime.fromtimestamp(date)
  new = orig + datetime.timedelta(days=7)
  new = new.timestamp()    
  #find the most recent price forward 1 week from story
  y2 = prices.loc[prices['date'] == new, stock]
  i=0
  while( len(y2)==0 and i < 250000 ):
    new = round((new + 100),-2)
    y2 = prices.loc[prices['date'] == new, stock]
    i = i + 1
  if(len(y2)==0):
    return 
  pc = ((float(y2)-float(y1))/float(y1))*100
  return pc



In [ ]:



def get_7_day_percent_change( date, stock):  
  if(math.isnan(date) or not stock in prices ):
    return
  date = int(math.modf(date/86400)[1]*86400)
  orig = datetime.datetime.fromtimestamp(date)
  new = orig + datetime.timedelta(days=7)
  new=int(new.timestamp())
  if( date < int(prices['date'].min()) or int(new) > prices['date'].max() ):
    return
  y1 = prices.loc[prices['date'] == date, stock]
  y2 = prices.loc[prices['date'] == new, stock]
  i=0
  while(len(y1)==0 and i < 24*30*3600 ):
    date = date - i
    print(i,date)
    y1 = prices.loc[prices['date'] == date, stock]
    i = i + 10
  i=0
  while(len(y2)==0 and i < 24*30*3600 ):
    new = new + i
    y2 = prices.loc[prices['date'] == new, stock]
    i = i + 10
  if( len(y1)==0 or len(y2==0) ):
    return
  else:
    print(date, new, y1, y2,  )
    return




Pandas Apply:   0%|          | 0/10 [00:00<?, ?it/s]

0 1643760000
10 1643759990
20 1643759970
30 1643759940
40 1643759900
50 1643759850
60 1643759790
70 1643759720
80 1643759640
90 1643759550
100 1643759450
110 1643759340
120 1643759220
130 1643759090
140 1643758950
150 1643758800
160 1643758640
170 1643758470
180 1643758290
190 1643758100
200 1643757900
210 1643757690
220 1643757470
230 1643757240
240 1643757000
250 1643756750
260 1643756490
270 1643756220
280 1643755940
290 1643755650
300 1643755350
310 1643755040
320 1643754720
330 1643754390
340 1643754050
350 1643753700
360 1643753340
370 1643752970
380 1643752590
390 1643752200
400 1643751800
410 1643751390
420 1643750970
430 1643750540
440 1643750100
450 1643749650
460 1643749190
470 1643748720
480 1643748240
490 1643747750
500 1643747250
510 1643746740
520 1643746220
530 1643745690
540 1643745150
550 1643744600
560 1643744040
570 1643743470
580 1643742890
590 1643742300
600 1643741700
610 1643741090
620 1643740470
630 1643739840
640 1643739200
650 1643738550
660 1643737890
670 16

KeyboardInterrupt: ignored

# Pass 2 at price data


In [ ]:
motley_7 = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/MA5851/assessments/A2/motley_8.csv')

In [ ]:

for index, story in motley_7[1:10].iterrows():
  print(story['time'])

1660631746.0
1660627203.0
1660610462.0
1660293028.0
1660199797.0
1660107662.0
1659996900.0
1659682293.0
1659681692.0


In [ ]:
for story in motley_7:
  print(story)

Unnamed: 0
company
article_url
category
pub_time
title
time
key_points
body


In [ ]:
motley_7['pc']=''

In [ ]:
len(motley_7) - motley_7['pc'].isna().sum()

23385

In [ ]:
motley_7['pc'][30890:30910]

30890   -9.07
30891   -9.07
30892    1.95
30893   -1.50
30894   -0.38
30895    0.49
30896   -1.85
30897    5.79
30898    1.66
30899    1.66
30900   -3.66
30901     NaN
30902     NaN
30903     NaN
30904     NaN
30905     NaN
30906     NaN
30907     NaN
30908     NaN
30909     NaN
Name: pc, dtype: float64

In [ ]:
url = 'https://au.finance.yahoo.com/quote/{}/history?period1={}&period2={}&interval=1d&filter=history&frequency=1d&includeAdjustedClose=true'

headers = { 'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/71.0.3578.98 Safari/537.36' }

soup = dict()

articles = pd.DataFrame( columns=['company', 'article_url', 'pub_time'])
errors = pd.DataFrame( columns=['company', 'script'] )
a_prices = pd.DataFrame()

for index, story in motley_7[600:615].iterrows():
  if(story['time']=='nan'):
    continue
  company = str(story['company']) + '.AX'
  start = int(story['time'])
  end = int(start + (60*60*24*7))
  page = url.format(company,start,end)
  print(page)
  print(story)
  response = requests.get(page, headers=headers)
  soup = BeautifulSoup(response.text, 'html.parser')
  pattern = re.compile(r'\s--\sData\s--\s')
  script_data=soup.find('script', text=pattern).contents[0]
  data = json.loads(script_data[112:-12])
  try:    
    s_prices = data['context']['dispatcher']['stores']['HistoricalPriceStore']['prices']
    s_prices = pd.DataFrame.from_dict(s_prices, orient='columns')   
    s_prices.set_index(s_prices['date'], drop=True, append=False, inplace=True, verify_integrity=False)    
    s_prices.drop(['date','open','high','low','volume','adjclose'], axis=1, inplace=True)
    pc = (float(s_prices['close'][s_prices.index[0]]) - float(s_prices['close'][s_prices.index[-1]]))/float(s_prices['close'][s_prices.index[-1]])
    print( s_prices['close'][s_prices.index[0]], s_prices['close'][s_prices.index[-1]], round(pc*100,2) )
  except:
    continue
      

https://au.finance.yahoo.com/quote/ABC.AX/history?period1=1623217579&period2=1623822379&interval=1d&filter=history&frequency=1d&includeAdjustedClose=true
company                                                      ABC
article_url    https://www.fool.com.au/2021/06/09/china-looki...
category                                       Share Market News
pub_time                                                     NaN
title          China looking to impose new controls on the ro...
time                                                1623217579.0
key_points                                                   NaN
body           China is considering imposing price controls t...
pc                                                          3.12
Name: 600, dtype: object
3.299999952316284 3.200000047683716 3.12
https://au.finance.yahoo.com/quote/ABC.AX/history?period1=1622615622&period2=1623220422&interval=1d&filter=history&frequency=1d&includeAdjustedClose=true
company                                  

In [ ]:
multiprocessing.cpu_count()

2

In [ ]:
url = 'https://au.finance.yahoo.com/quote/{}/history?period1={}&period2={}&interval=1d&filter=history&frequency=1d&includeAdjustedClose=true'
headers = { 'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/71.0.3578.98 Safari/537.36' }
soup = dict()

articles = pd.DataFrame( columns=['company', 'article_url', 'pub_time'])
errors = pd.DataFrame( columns=['company', 'script'] )
a_prices = pd.DataFrame()

for index, story in tqdm(motley_7[30901:].iterrows(), total = len(motley_7[30901:])):
  company = str(story['company']) + '.AX'
  try:
    start = int(story['time'])
  except:
    continue
  end = int(start + (60*60*24*7))
  page = url.format(company,start,end)
  response = requests.get(page, headers=headers)
  soup = BeautifulSoup(response.text, 'html.parser')
  pattern = re.compile(r'\s--\sData\s--\s')
  try:    
    script_data=soup.find('script', text=pattern).contents[0]
    data = json.loads(script_data[112:-12]) 
    s_prices = data['context']['dispatcher']['stores']['HistoricalPriceStore']['prices']
    s_prices = pd.DataFrame.from_dict(s_prices, orient='columns')   
    s_prices.set_index(s_prices['date'], drop=True, append=False, inplace=True, verify_integrity=False)    
    s_prices.drop(['date','open','high','low','volume','adjclose'], axis=1, inplace=True)
    pc = (float(s_prices['close'][s_prices.index[0]]) - float(s_prices['close'][s_prices.index[-1]]))/float(s_prices['close'][s_prices.index[-1]])
   
    motley_7.loc[index, 'pc'] = round(pc*100,2)
  except:
    continue
  if(int(index) % 100 == 0):
    motley_7.to_csv('/content/drive/MyDrive/Colab Notebooks/MA5851/assessments/A2/motley_9.csv')

  0%|          | 0/11747 [00:00<?, ?it/s]

In [ ]:
motley_7.to_csv('/content/drive/MyDrive/Colab Notebooks/MA5851/assessments/A2/motley_9.csv')

In [ ]:
motley_7['pc'][30890:30910]

,company,article_url,category,pub_time,title,time,key_points,body,pc
0,360,https://www.fool.com.au/2022/08/17/down-40-thi...,Technology Shares,NaN,"Down 40% this year, should you pounce on this ...",1.660695e+09,NaN,The Life360 Inc (ASX: 360) share price swung l...,-13.9
1,360,https://www.fool.com.au/2022/08/16/here-are-th...,Share Market News,NaN,Here are the top 10 ASX 200 shares today,1.660632e+09,NaN,S&P/ASX 200 Index (ASX: XJO) shares pushed thr...,-11.72
2,360,https://www.fool.com.au/2022/08/16/why-bhp-lif...,Share Gainers,NaN,"Why BHP, Life360, Tassal, and Temple & Webster...",1.660627e+09,NaN,"In afternoon trade, the S&P/ASX 200 Index (ASX...",-11.72
3,360,https://www.fool.com.au/2022/08/16/life360-sha...,Earnings Results,NaN,Life360 share price rises as revenue more than...,1.660610e+09,The Life360 share price is climbing 3% higher ...,The Life360 Inc (ASX: 360) share price is boun...,-11.72
4,360,https://www.fool.com.au/2022/08/12/2-exciting-...,Growth Shares,NaN,2 exciting ASX tech shares to buy now accordin...,1.660293e+09,NaN,"If you’re searching for growth shares to buy, ...",4.26
5,360,https://www.fool.com.au/2022/08/11/here-are-th...,Share Market News,NaN,Here are the top 10 ASX 200 shares today,1.660200e+09,NaN,The S&P/ASX 200 Index (ASX: XJO) regained its ...,6.88
6,360,https://www.fool.com.au/2022/08/10/whats-going...,Technology Shares,NaN,What’s going so wrong for ASX 200 tech shares ...,1.660108e+09,NaN,It’s been a mildly negative day of trading for...,24.21
7,360,https://www.fool.com.au/2022/08/09/here-are-2-...,Technology Shares,NaN,Here are 2 excellent ASX tech shares a top bro...,1.659997e+09,NaN,With the tech sector still down materially sin...,7.63
8,360,https://www.fool.com.au/2022/08/05/these-3-asx...,Technology Shares,NaN,These 3 ASX tech shares had a cracking day on ...,1.659682e+09,NaN,It ended up being a very pleasing end to the w...,8.87
9,360,https://www.fool.com.au/2022/08/05/here-are-th...,Share Market News,NaN,Here are the top 10 ASX 200 shares today,1.659682e+09,NaN,The S&P/ASX 200 Index (ASX: XJO) finished the ...,8.87
